In [1]:
import pandas as pd
import numpy as np
import time
import os

t = time.localtime()
timestamp = time.strftime('%y%m%d-%H%M', t)

# read excel
df = pd.read_excel('H://Current_Working_Folder//Reports Analytics//202108//Version3_psycopg//202108-t_worc_families-.xlsx')

# assign path and reporitng month
drive = 'H://Current_Working_Folder//Reports Analytics//'
reporting_month = str(df.loc[0,'reporting_month'])

# Filter with column names ending with amt, amount, nbr_month
df_amt = df.filter(regex = 'amt$|amount$|amount$|nbr_month$|nbr_months', axis='columns')

# Describe the filtered dataframe
df_des = df_amt.describe()

# Write the describe to excel
writer = pd.ExcelWriter(drive + reporting_month + '//' + reporting_month + '_' + 'report' + '-' + timestamp + '.xlsx')
#df_des.to_excel(writer, index = False)
df_des.to_excel(writer, sheet_name='family_stat')
writer.save()

In [2]:
#df.columns

In [3]:
df.shape

(1397, 56)

In [4]:
df['subsid_child_care'].value_counts()

No subsidized child care received                                                                                                                 1315
Yes, receives child care funded entirely or in part with Federal funds (e.g., receives TANF, CCDF, SSBG, or other federally funded child care)      82
Name: subsid_child_care, dtype: int64

In [5]:
df['subsid_child_care_amt'].sum()

54968

In [6]:
df['tanf child care_children_covered'].value_counts()

0    1397
Name: tanf child care_children_covered, dtype: int64

In [7]:
df['tanf child care_amount'].sum()

0

In [8]:
cat = df.drop(df_amt, axis='columns')
#cat1 = cat.drop((cat.loc[:,'reporting_month':'stratum']),axis='columns')
cat1 = cat.drop(['id','reporting_month','case_number','county_fips_code','stratum','disposition','worker_id',
                 'case_manager','area','created_by','updated_by','created_at','updated_at','ans_county',
                 'local_office','waiver_grps'], axis='columns')
cat2 = cat1.drop(cat1.filter(regex = '^reason|^other'), axis='columns')

In [9]:
reason_cat = cat1.filter(regex = '^reason', axis='columns')

for c in reason_cat.columns:
    print(reason_cat[c].value_counts())
    print('---')


No     1242
Yes     155
Name: reason for amount of reduction_work requirements sanction, dtype: int64
---
No    1397
Name: reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent, dtype: int64
---
No    1397
Name: reason for amount of reduction_Sanction for Teen Parent not Attending School, dtype: int64
---
No     978
Yes    419
Name: reason for amount of reduction_Non-Cooperation with Child Support, dtype: int64
---
No     1389
Yes       8
Name: reason for amount of reduction_Failure to Comply with an Individual Responsibility Plan, dtype: int64
---
No     1378
Yes      19
Name: reason for amount of reduction_Other Sanction, dtype: int64
---
0    1397
Name: reason for amount of reduction_Recoupment of Prior Overpayment, dtype: int64
---


In [10]:
#((df.filter(regex = '^reason'))).head(20)

In [11]:
for c in cat2.columns:
    print((cat2[c].value_counts().to_frame('count').rename_axis(c)).reset_index())
    print('---')

   category  count
0       Tea   1350
1  WorkPays     47
---
     zip_code  count
0       71854     52
1       72301     43
2       72209     40
3       72401     39
4       72390     34
..        ...    ...
284     72722      1
285     72705      1
286     71642      1
287     72734      1
288     72556      1

[289 rows x 2 columns]
---
                                      funding_stream  count
0  Funded, in whole or in part, with Federal TANF...   1397
---
                       new_applicant  count
0     Not newly-approved application   1313
1  Yes, a newly-approved application     84
---
   family_size  count
0            2    710
1            3    398
2            4    187
3            5     65
4            6     30
5            7      6
6           10      1
---
                                         family_type  count
0            Family with no work-eligible individual    822
1  Family included only in overall work participa...    546
2  Two-Parent Family included in both t

In [12]:
worc_fam = df
fam_type = worc_fam.groupby(['category','family_type']).agg({'case_number' : 'nunique'}).rename({'case_number': 'case_count'}, axis='columns')#.reset_index()
fam_type

case_count
category family_type                                                   
Tea      Family included only in overall work participat...         502
         Family with no work-eligible individual                    822
         Two-Parent Family included in both the overall ...          26
WorkPays Family included only in overall work participat...          44
         Two-Parent Family included in both the overall ...           3